In [1]:
library(topGO)
library(tidyverse)

Loading required package: BiocGenerics
Loading required package: parallel

Attaching package: ‘BiocGenerics’

The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB

The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs

The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which, which.max, which.min

Loading required package: graph
Loading required package: Biobase
Welcome to Bioconductor

    Vignettes contain i

In [2]:
all_ann <- read_csv("all_ann.csv")

Warning message:
“Missing column names filled in: 'X1' [1]”Parsed with column specification:
cols(
  .default = col_character(),
  X1 = col_double(),
  index = col_double(),
  length = col_double(),
  start = col_double(),
  stop = col_double(),
  status = col_logical()
)
See spec(...) for full column specifications.


### Make association file following this structure

`gene_ID<TAB>GO_ID1, GO_ID2, GO_ID3, ....`
 
    

In [3]:
geneID2GO <- readMappings("go_mapping.csv")
str(head(geneID2GO))

List of 6
 $ protein_accession: chr "go"
 $ XP_018740815_1   : chr [1:2] "GO:0005887" "GO:0055085"
 $ XP_018741507_1   : chr "GO:0005515"
 $ XP_018741507_1   : chr "GO:0005515"
 $ XP_018741507_1   : chr "GO:0005515"
 $ XP_018741507_1   : chr "GO:0005515"


In [4]:
GO2geneID <- inverseList(geneID2GO)

In [5]:
str(head(GO2geneID))

List of 6
 $ go        : chr "protein_accession"
 $ GO:0000001: chr [1:44] "XP_018740907_1" "XP_018740907_1" "XP_018740907_1" "XP_018740907_1" ...
 $ GO:0000002: chr [1:57] "SJX65437_1" "SJX65437_1" "SJX64063_1" "XP_016272549_1" ...
 $ GO:0000009: chr [1:22] "SJX62846_1" "XP_016270873_1" "XP_014565336_1" "KIR69017_1" ...
 $ GO:0000015: chr [1:248] "XP_018738952_1" "XP_018738952_1" "XP_018738952_1" "XP_018738952_1" ...
 $ GO:0000027: chr [1:71] "XP_018740599_1" "XP_018741865_1" "SJX60355_1" "SJX65482_1" ...


Loop through files to get the names of the species

In [6]:
taxa<-gsub("\\.faa$","", list.files("faas","*.faa"))

In [7]:
taxa

[1] "Cryptococcus_gattii"        "Cryptococcus_neoformans"   
 [3] "Fomitiporia_mediterranea"   "Malassezia_globosa"        
 [5] "Malassezia_restricta"       "Malassezia_sympodialis"    
 [7] "Melampsora_larici-populina" "Mixia_osmundae"            
 [9] "Moniliophthora_roreri"      "Paxillus_involutus"        
[11] "Piloderma_croceum"          "Puccinia_striiformis"      
[13] "Puccinia_triticina"         "Rhizoctonia_solani"        
[15] "Rhodotorula_toruloides"     "Serendipita_indica"        
[17] "Sporisorium_reilianum"      "Trichosporon_asahii"       
[19] "Ustilago_bromivora"         "Ustilago_hordei"           
[21] "Ustilago_maydis"

In [9]:
species_name <- 'Cryptococcus_gattii'

In [10]:
all_achy <- all_ann %>%
filter(family == species_name) %>%
select(3)%>%
distinct(protein_accession) %>%
as.matrix %>%
as.vector
names(all_achy) <- all_achy

In [12]:
sel_achy <- all_ann %>%
filter(family == species_name) %>%
filter(ps == "yes") %>%
select(3) %>%
as.matrix %>%
as.vector

In [19]:
all_ann %>%
filter(ps=="yes") %>%
filter(family == "Cryptococcus_gattii")

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
“number of rows of result is not a multiple of vector length (arg 2)”

X1,index,protein_accession,md5,length,analysis,signature_accession,signature_description,start,stop,⋯,status,date,ip_accesion,ip_description,go,pathway,family,ps,dups,signal
<dbl>,<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,⋯,<lgl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>


In [8]:
result_list = list()

for(species_name in taxa) {
all_achy <- all_ann %>%
filter(family == species_name) %>%
select(3)%>%
distinct(protein_accession) %>%
as.matrix %>%
as.vector
names(all_achy) <- all_achy

sel_achy <- all_ann %>%
filter(family == species_name) %>%
filter(ps == "yes") %>%
select(3) %>%
as.matrix %>%
as.vector

names(sel_achy) <- sel_achy
geneList <- factor(as.integer (all_achy %in% sel_achy))
names(geneList) <- all_achy 

achyGOdata <- new("topGOdata",
                    description = "achy without p-values", ontology = "BP",
                    a = geneList,
                    nodeSize = 5,
                    annot = annFUN.gene2GO, gene2GO = geneID2GO)
test.ks <- new("classicScore", testStatistic = GOKSTest, name = "KS tests")
achyKS <- getSigGroups(achyGOdata, test.ks)
test.elim <- new("elimScore", testStatistic = GOKSTest, name = "Fisher test", cutOff = 0.01)

achyElim <- getSigGroups(achyGOdata, test.elim)
test.weight <- new("weightCount", testStatistic = GOFisherTest, name = "Fisher test", sigRatio = "ratio")

achyWeight <- getSigGroups(achyGOdata, test.weight)

achyFis <- runTest(achyGOdata, algorithm = "weight01", statistic = "fisher")

achyRes <- GenTable(achyGOdata, classic = achyFis, KS = achyKS, weight = achyWeight,                    
                   orderBy = "weight", ranksOf = "classic", topNodes = 50)
achyRes$genes <- sapply(achyRes$GO.ID, function(x)
    {
      genes<-genesInTerm(achyGOdata, x) 
      genes[[1]][genes[[1]] %in% sel_achy]
    })
result_list[[species_name]] = achyRes
# graph
pValue.classic <- score(achyKS)
pValue.elim <- score(achyElim)[names(pValue.classic)]
gstat <- termStat(achyGOdata, names(pValue.classic))
gSize <- gstat$Annotated / max(gstat$Annotated) * 4
#gCol <- colMap(gstat$Significant)
plot(pValue.classic, pValue.elim, xlab = "p-value classic", ylab = "p-value elim",
    pch = 19, cex = gSize)
#
}


ERROR: Error in .local(.Object, ...): allGenes must be a factor with 2 levels


In [13]:
sel_achy <- all_ann %>%
filter(family == species_name) %>%
filter(ps == "yes") %>%
select(3) %>%
as.matrix %>%
as.vector